# Load triples to HANA Cloud KG Engine

In [2]:
#Credentials
address='xxx'
port='443'
user='DBADMIN'
password='xxx'

In [3]:
from hdbcli import dbapi

# connect to database using username/password
conn = dbapi.connect(user=user, password=password, address=address, port=port)
cursor = conn.cursor()

In [47]:
#Populate a new RDF in HANA Cloud with the ttl file content. We put the ontology into a specific graph.
path = "xxx"
ttl_filename = path + "ontology.ttl"
graph_name = "http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test"

In [41]:
# Drop existing one if needed
query = """drop graph <{graph_name}>""". format(graph_name=graph_name)
print(query)

drop graph <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test>


In [25]:
#Uncomment to drop the ontology
#resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, '', '?', None) )

In [48]:
with open(ttl_filename, 'r') as ttlfp:
    request_hdrs = ''
    request_hdrs += 'rqx-load-protocol: true' + '\r\n'            # required header for upload protocol
    request_hdrs += 'rqx-load-filename: ' + ttl_filename + '\r\n' # optional header
    request_hdrs += 'rqx-load-graphname: ' + graph_name + '\r\n'   # optional header to specify name of the graph, 
                                                                  #if not provided RDF data will be loaded to internal-default-graph
    conn.cursor().callproc('SPARQL_EXECUTE', (ttlfp.read(), request_hdrs, '?', None))

In [51]:
query = """
    SELECT * 
    FROM SPARQL_TABLE('
        prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/>
        select ?s 
        from <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test> 
        where { 
            ?s a :SAPEmployee
            }
        '
        )"""

In [52]:
#There is just one individual
cursor.execute(query)
for row in cursor:
    print(row)

('http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology#xxxx.yyyyy@sap.com',)


In [38]:
#Function to load more ttl files
def load_file(ttl_filename, graph_name):
    try:
        with open(ttl_filename, 'r') as ttlfp:
            request_hdrs = ''
            request_hdrs += 'rqx-load-protocol: true' + '\r\n'            # required header for upload protocol
            request_hdrs += 'rqx-load-filename: ' + ttl_filename + '\r\n' # optional header
            request_hdrs += 'rqx-load-graphname: ' + graph_name + '\r\n'  # optional header to specify name of the graph, 
                                                                          # if not provided RDF data will be loaded 
                                                                          # to internal-default-graph
            conn.cursor().callproc('SPARQL_EXECUTE', (ttlfp.read(), request_hdrs, '?', None))
        return 0
    except:
        print("An exception occurred")
        return 1

In [39]:
query = """drop graph <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>"""

In [258]:
#Uncomment to drop the rdf
#resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, '', '?', None) )

In [53]:
import glob

path = "xxx"

list = sorted(glob.glob(path + "*.ttl"))
print(list[5])
print(len(list))

/Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/INDUSTRY2PSR.ttl
29
range(0, 29)


In [54]:
for i in range(len(list)):
    print(i, list[i])
    ttl_filename = list[i]
    graph_name = "http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test"
    load_file(ttl_filename, graph_name)

0 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/AI_SCENARIO_TYPE.ttl
1 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/BTP_SCENARIO_TYPE.ttl
2 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/COUNTRY.ttl
3 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/GTM_OUTCOME.ttl
4 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-and-reasoning/knowledge_graph/instances/GTM_ROUTE.ttl
5 /Users/I559573/Documents/GitHub/btp-generative-ai-hub-use-cases/51-Knowledge-Graph-Explicit-knowledge-representation-a

In [67]:
# Just check the scenario type URI
query = """
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/>
    select ?s 
    from <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    where {
        #?s a :SAPAIScenario
        #?s a :UseCase
        ?s a :SAPPartnerContact
        }
    LIMIT 10
"""

In [68]:
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

s
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/oliver.lee@362109.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/noah.brown@1719068.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/olivia.patel@708379.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/ava.thomas@698634.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/ella.rodriguez@481405.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/liam.wilson@1691507.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/amelia.wilson@663962.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/lucas.johnson@665751.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/isabella.clark@515403.com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/oliver.brow

In [93]:
# Check all the service requests initiated by the partner, in particular those from Partner2301317
query = """
    SELECT * 
    FROM SPARQL_TABLE('
        prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
        prefix owl: <http://www.w3.org/2002/07/owl#> 
        
        select ?psrRef ?partnerName ?initiatorValue ?status
        from <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test> 
        where { 
            ?psrRef a :SAPServiceRequest ;
                :wasInitiatedBy ?initiatorRef ;
                :requestedBy ?partnerRef ;
                :hasStatus ?status .
            ?partnerRef :accountName ?partnerName .
            ?initiatorRef :hasName ?initiatorValue .
            }
        LIMIT 10
        '
    )"""

In [94]:
cursor.execute(query)
for row in cursor:
    print(row)

('http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/ServiceRequest85ef55b1-e74a-4be5-b7e3-835873ffcb06', 'Hogwarts Consulting', 'Partner', 'completed')


In [71]:
# Count the number of triples in the rdf
query = """
    SELECT (COUNT(*) as ?Triples) 
    WHERE 
      { GRAPH <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test> 
          { ?s ?p ?o } 
      }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

Triples
54626



In [74]:
# Obtain a list of classes in the ontology
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    
    SELECT DISTINCT ?c
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test>
    WHERE {
        ?c a owl:Class
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

c
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPServiceRequest
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPBTPScenario
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPEmployee
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/GTMOutcome
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPPartnerType
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPPartner
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/RequiredSupport
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SourceOfRequest
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPServiceRequestInitiator
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPProgramInitiative
http://www.semanticweb.org/ontologies/2025/smart-technical-advisor

In [75]:
# Obtain a list of object properties in the ontology
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    
    SELECT DISTINCT ?p
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test>
    WHERE {
        ?p a owl:ObjectProperty
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

p
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/requestedBy
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/involves
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/aiScenarioType
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/belongsToIndustry
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/consistsOf
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasGTMOutcome
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasPartnerType
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasPartnershipType
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/modifiedBy
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/relatesToLoB
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/responsiblePer

In [76]:
# Obtain a list of datatype properties in the ontology
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#> 
    
    SELECT DISTINCT ?p
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test>
    WHERE {
        ?p a owl:DatatypeProperty
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

p
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/accountName
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/countryCode
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/createdOnDate
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasCustomer
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasID
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasPBCOrderNumber
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasStatus
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/isAIUseCase
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/solutionSubArea
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/useCaseStatus
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasName
http://www.semant

In [105]:
# Look for a certain partner in a certain region
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#> 

    SELECT ?partnerRef ?partnerName ?regionName
    	FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    	WHERE {
        	?partnerRef a :SAPPartner ; 
                :accountName ?partnerName ;
                :ofCountry ?countryRef ;
                :partnerStatus 'Active' .
            ?countryRef :belongsToSAPRegion ?regionRef .
            ?regionRef :hasName ?regionName .
            FILTER regex(?partnerName, "Hogwarts", "i") .
            FILTER regex(?regionName, "Middle & Eastern Europe", "i") .
    }
    LIMIT 100
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

partnerRef,partnerName,regionName
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/Partner2310370,Hogwarts Consulting,Middle & Eastern Europe



In [117]:
# Find all the service requests where the service was a TA and show who delivered the service
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>

    SELECT ?psrRef ?serviceRef ?serviceTypeName ?productName ?sapPersonName
    	FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    	WHERE {
            ?serviceRef a :SAPService ;   
                :relatesToServiceRequest ?psrRef ;
                :serviceType ?serviceTypeRef ;
                :deliveredBy ?sapPersonRef .
            ?sapPersonRef foaf:name ?sapPersonName .
            ?serviceTypeRef :hasName ?serviceTypeName .
            ?psrRef :coversProduct ?productRef .
            ?productRef :productName ?productName .
    }
    LIMIT 10
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

psrRef,serviceRef,serviceTypeName,productName,sapPersonName
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/ServiceRequest85ef55b1-e74a-4be5-b7e3-835873ffcb06,http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/Service39b8384b-8924-4338-9677-8082eef59ae8,SAP BTP Technical Advisory,Business One,John Legend



In [104]:
#ASK Returns a boolean indicating whether a query pattern matches or not.
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>
    
    ASK
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    WHERE {
    	?sapPersonRef a :SAPEmployee ;
            foaf:member :SAPOrg1 .
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

true


In [112]:
# SAP employes that are also BTP SA
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>
    
    SELECT *
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    WHERE {
    	?sapPersonRef a :SAPEmployee ;
            foaf:member :SAPOrg1 .
    }
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

sapPersonRef
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/john.legend@sap.com



In [122]:
# Find how many TAs have been delivered by each BTP SA
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>

    SELECT 
            ?sapPersonName 
            (COUNT(?psrRef) AS ?psrCount) (COUNT(?serviceRef) AS ?serviceCount)
            (COUNT(?btpRef) AS ?btpCount) (COUNT(?aiRef) AS ?aiCount)
    	FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-rdf-test>
    	WHERE {
            ?serviceRef a :SAPService ;
                :serviceType ?serviceTypeRef ;
                :relatesToServiceRequest ?psrRef ;
                :deliveredBy ?sapPersonRef .
            ?sapPersonRef foaf:name ?sapPersonName ;
                foaf:member :SAPOrg1 .
            ?psrRef :hasUseCase ?useCaseRef .
            OPTIONAL 
                {?useCaseRef :btpScenarioType ?btpRef .}
            OPTIONAL 
                {?useCaseRef :aiScenarioType ?aiRef .}
            ?serviceTypeRef :hasName ?serviceTypeName .            
            FILTER regex(?serviceTypeName, "SAP BTP Technical Advisory", "i") .
    } GROUP BY ?sapPersonName
    ORDER BY ?sapPersonName
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2]) 

sapPersonName,psrCount,serviceCount,btpCount,aiCount
John Legend,1,1,1,0



## Other queries

In [134]:
# Get the onthology schema 
query = """
    CONSTRUCT {?s ?p ?o} FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test> 
    WHERE {?s ?p ?o}
    LIMIT 10
"""
resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2])

@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
<http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasUseCase> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#FunctionalProperty> .
<http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/relatesToServiceRequest> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#FunctionalProperty> .
<http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPServiceRequest> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Class> .
<http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasUseCase> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#ObjectProperty> .
<http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPBTPScenario> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#

In [135]:
#Get properties and their comments
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>
    
    SELECT DISTINCT ?op ?com 
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test> 
    WHERE { 
        {?op rdf:type owl:ObjectProperty .}
        UNION
        {?op rdf:type owl:DatatypeProperty .}
        ?op rdfs:comment ?com .
    }
    LIMIT 10
"""

resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2])

op,com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasPBCOrderNumber,It indicates the unique  PBC order number of the service (if the service was requested through the Parter Benefits Catalog).
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasName,This property indicates the proper name of an object.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/requestDate,It is a property of the service request class. It indicates the date the service request was submitted.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/involves,It indicates the service request or service involves a certain person.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/accountName,It specifies the account name of the SAP partner.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/hasCustomer,It marks whether the use case has already a customer or 

In [136]:
#Get classes and their comments
query = """
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix : <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/> 
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix foaf:  <http://xmlns.com/foaf/0.1/>

    SELECT DISTINCT ?cls ?com
    FROM <http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology-test> 
    WHERE { 
        ?cls rdf:type owl:Class . 
        FILTER (isIRI(?cls)) . 
        OPTIONAL { ?cls rdfs:comment ?com } 
    }
    LIMIT 10
"""

resp = conn.cursor().callproc('SPARQL_EXECUTE', (query, 'Accept: application/sparql-results+csv', '?', None) )
print(resp[2])

cls,com
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPBTPScenario,"Possible scenario when implementing a solution on SAP BTP. It can be one of the following:
'Data-to-Value'
'Extensibility'
'Integration'"
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/GTMOutcome,Outcome of the Go-to-Market process if there is one related to the service request submitted.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/RequiredSupport,Type of support required for the service.
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SourceOfRequest,"Channels through which the request can arrive. It can be:
'PCD E-Mail'
'PBC request'
'Internal'
'Others'"
http://www.semanticweb.org/ontologies/2025/smart-technical-advisory-ontology/SAPServiceRequestInitiator,"Who initiated the service request. It can be:
'SAP'
'Customer'
'Partner'"
http://www.semanticweb.org/ontologies/2025/smart-technical-adviso